# Story Generation
We remember things better as stories. The plan here is to pick phrases from a pack we have learnt, and incoprate them into a story.

In [1]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.llm_tools.story_generation import generate_story
from src.phrases.search import get_phrases_by_collection
from src.utils import save_json, load_json
from src.story import Story

In [11]:
COLLECTION = "LM2000"
DECK = "Pack05"

In [12]:
phrase_list = get_phrases_by_collection(COLLECTION, DECK)

In [13]:
story_name, summary, story_dialogue = generate_story(
    phrase_list=phrase_list,
    learner_level="intermediate",
    story_length="45 seconds",
)

In [14]:
print(story_name)  #
print(summary)

The Winter Restaurant Challenge
Alex and Sam must handle a difficult situation at their restaurant when winter approaches and a food critic makes a bold claim that affects their business.


In [15]:
save_json(
    story_dialogue, f"../data/stories/{COLLECTION}-{DECK}-{story_name}-story.json"
)

In [16]:
story_dialogue = load_json(
    f"../data/stories/{COLLECTION}-{DECK}-{story_name}-story.json"
)

In [17]:
story = Story.create(
    title=story_name,
    summary="Intermediate story. " + summary,
    story_dialogue=story_dialogue,
    collection=COLLECTION,
    deck=DECK,
)

In [18]:
# story.translate("en-GB", overwrite=True)
# story.upload()
# story.publish_story(source_language="en-GB", target_language="cmn-CN")
story.publish_story(source_language="en-GB", target_language="sv-SE")

2026-02-15 19:08:52 - audio-language-trainer - WARNING - nlp.py:42 - analyze_text_syntax: language not supported by Google NLP: sv
2026-02-15 19:08:52 - audio-language-trainer - INFO - phrase_model.py:337 - (y) Created translation for sv-SE: Sam, vintern närmar sig snabbt. Vi behöver göra några ändringar på restaurangmenyn.
2026-02-15 19:08:53 - audio-language-trainer - WARNING - nlp.py:42 - analyze_text_syntax: language not supported by Google NLP: sv
2026-02-15 19:08:53 - audio-language-trainer - INFO - phrase_model.py:337 - (y) Created translation for sv-SE: Du har rätt. Vad rekommenderar du? Våra kunder har bett om mer soppa och varma måltider.
2026-02-15 19:08:54 - audio-language-trainer - WARNING - nlp.py:42 - analyze_text_syntax: language not supported by Google NLP: sv
2026-02-15 19:08:54 - audio-language-trainer - INFO - phrase_model.py:337 - (y) Created translation for sv-SE: Jag tänkte att vi kunde servera en ny svampsoppa med färskt bröd. Jag gjorde den igår och den smakar 

'https://storage.googleapis.com/audio-language-trainer-stories/stories/en-GB/sv-SE/the_winter_restaurant_challenge_c5056c/index.html'

In [19]:
story.key

'the_winter_restaurant_challenge_c5056c'